In [6]:
# TODO: とりあえず使えそうなanimeのカラムを使える形にする → Baselineに追加してみる
# TODO: 12th place solutionにあげられている特徴量エンジニアリングを行う → Baselineに追加してみる
# TODO: node2vec特徴量を作成 → Baselineに追加してみる
# TODO: seen, unseenでユーザーを分けてモデル作成
# TODO: seen:stratifiedKFold, unseen:GroupKFoldでCV
# TODO: seenでしか作れない特徴量エンジニアリング


In [3]:
import polars as pl

from src.data import load_data

pl.Config.set_fmt_str_lengths(100)
pl.Config.set_tbl_rows(20)
pl.Config.set_tbl_cols(20)


polars.config.Config

In [4]:
train, test, anime = load_data()
anime.head()


anime_id,genres,japanese_name,type,episodes,aired,producers,licensors,studios,source,duration,rating,members,watching,completed,on_hold,dropped,plan_to_watch
str,str,str,str,str,str,str,str,str,str,str,str,i32,i32,i32,i32,i32,i32
"""000ba7f7e34e107e7544""","""Comedy, Sci-Fi, Seinen, Slice of Life, Space""","""宇宙兄弟""","""TV""","""99""","""Apr 1, 2012 to Mar 22, 2014""","""Aniplex, Dentsu, YTV, Trinity Sound""","""Sentai Filmworks""","""A-1 Pictures""","""Manga""","""24 min. per ep.""","""PG-13 - Teens 13 or older""",150428,16552,37234,13009,6948,76685
"""00427279d72064e7fb69""","""Adventure, Slice of Life, Mystery, Historical, Supernatural, Fantasy, Seinen""","""蟲師""","""TV""","""26""","""Oct 23, 2005 to Jun 19, 2006""","""Avex Entertainment, Marvelous, SKY Perfect Well Think, Delfi Sound""","""Funimation""","""Artland""","""Manga""","""25 min. per ep.""","""PG-13 - Teens 13 or older""",620736,55482,235371,42786,20017,267080
"""00444b67aaabdf740a68""","""Adventure, Slice of Life, Mystery, Historical, Supernatural, Fantasy, Seinen""","""蟲師 続章""","""TV""","""10""","""Apr 5, 2014 to Jun 21, 2014""","""Aniplex, Kodansha, Delfi Sound""","""Aniplex of America""","""Artland""","""Manga""","""24 min. per ep.""","""PG-13 - Teens 13 or older""",226522,12585,113559,6095,2606,91677
"""00839a3507ab168abe75""","""Comedy, Ecchi, Fantasy, School""","""星刻の竜騎士""","""TV""","""12""","""Apr 5, 2014 to Jun 21, 2014""","""Media Factory, AT-X, Sony Music Communications, Tsukuru no Mori""","""Funimation""","""C-Station""","""Light novel""","""24 min. per ep.""","""R+ - Mild Nudity""",170220,8723,118202,3753,8034,31508
"""0192331235e110fe4f76""","""Comedy, Harem, Romance, Sci-Fi, Shounen, Space""","""天地無用！""","""TV""","""26""","""Apr 2, 1995 to Sep 24, 1995""","""TV Tokyo, Pioneer LDC""","""Funimation, Geneon Entertainment USA""","""AIC""","""Original""","""23 min. per ep.""","""PG-13 - Teens 13 or older""",62599,2565,39890,2093,1986,16065


In [6]:
anime.height


2000

In [21]:
anime["anime_id"].unique().len()


2000

* animeは全部で2000件、重複はない

In [ ]:
# TODO:genresは後回し


In [28]:
# japanese_name

# japanese_name カラムで重複どれだけあるか確認
duplicate_japanese_name_list = (
    anime.group_by("japanese_name").len().filter(pl.col("len") > 1)["japanese_name"].to_list()
)
print(len(duplicate_japanese_name_list))


59


In [30]:
# japanese_nameが重複してるアニメを確認
anime.filter(pl.col("japanese_name").is_in(duplicate_japanese_name_list)).sort("japanese_name")


anime_id,genres,japanese_name,type,episodes,aired,producers,licensors,studios,source,duration,rating,members,watching,completed,on_hold,dropped,plan_to_watch
str,str,str,str,str,str,str,str,str,str,str,str,i32,i32,i32,i32,i32,i32
"""1a16b40defc2b0fb4cf1""","""Action, Adventure, Comedy, Historical, Samurai, Fantasy, Seinen""","""DRIFTERS""","""TV""","""12""","""Oct 7, 2016 to Dec 23, 2016""","""Hoods Entertainment, NBCUniversal Entertainment Japan, Shounen Gahousha""","""Funimation""","""Hoods Drifters Studio""","""Manga""","""23 min. per ep.""","""R - 17+ (violence & profanity)""",410926,27263,252770,12026,15984,102883
"""959f7ac73fdf23559639""","""Action, Adventure, Comedy, Fantasy, Historical, Samurai, Seinen""","""DRIFTERS""","""OVA""","""2""","""Dec 23, 2017""","""Unknown""","""Unknown""","""Hoods Drifters Studio""","""Manga""","""26 min. per ep.""","""R - 17+ (violence & profanity)""",60741,1462,26092,686,418,32083
"""06b8f4e07020115e4e4d""","""Action, Adventure, Comedy, Magic, Fantasy, Shounen""","""FAIRY TAIL（フェアリーテイル）""","""TV""","""175""","""Oct 12, 2009 to Mar 30, 2013""","""TV Tokyo, Dentsu, Pony Canyon, Kodansha, DAX Production, Sakura Create""","""Funimation""","""Satelight, A-1 Pictures""","""Manga""","""24 min. per ep.""","""PG-13 - Teens 13 or older""",1312470,185463,737096,108697,148408,132806
"""97c09d33df70353ec064""","""Action, Adventure, Comedy, Fantasy, Magic, Shounen""","""FAIRY TAIL（フェアリーテイル）""","""TV""","""102""","""Apr 5, 2014 to Mar 26, 2016""","""TV Tokyo, Dentsu""","""Funimation""","""A-1 Pictures, Bridge""","""Manga""","""24 min. per ep.""","""PG-13 - Teens 13 or older""",716382,101344,440657,43729,50700,79952
"""2db8392531876c3c6326""","""Action, Adventure, Super Power, Fantasy, Shounen""","""HUNTER×HUNTER（ハンター×ハンター）""","""TV""","""62""","""Oct 16, 1999 to Mar 31, 2001""","""Fuji TV""","""VIZ Media""","""Nippon Animation""","""Manga""","""23 min. per ep.""","""PG-13 - Teens 13 or older""",416227,28061,260968,15535,15100,96563
"""9249d96310a6564d092c""","""Action, Adventure, Fantasy, Shounen, Super Power""","""HUNTER×HUNTER（ハンター×ハンター）""","""TV""","""148""","""Oct 2, 2011 to Sep 24, 2014""","""VAP, Nippon Television Network, Shueisha""","""VIZ Media""","""Madhouse""","""Manga""","""23 min. per ep.""","""PG-13 - Teens 13 or older""",1673924,227277,1094486,79587,31652,240922
"""d7cee297224130410c3b""","""Action, Adventure, Comedy, Super Power, Drama, Fantasy, Shounen""","""ONE PIECE""","""TV""","""Unknown""","""Oct 20, 1999 to ?""","""Fuji TV, TAP, Shueisha""","""Funimation, 4Kids Entertainment""","""Toei Animation""","""Manga""","""24 min.""","""PG-13 - Teens 13 or older""",1352724,887333,33,187919,136245,141194
"""eb25d8bf9c986ad62c41""","""Action, Adventure, Comedy, Super Power, Fantasy, Shounen""","""ONE PIECE""","""Movie""","""1""","""Mar 4, 2000""","""Unknown""","""Unknown""","""Toei Animation""","""Manga""","""50 min.""","""PG-13 - Teens 13 or older""",82437,1081,72673,408,357,7918
"""8e32ba5ae413a8a4857e""","""Psychological, Drama, Thriller, Fantasy""","""Re：ゼロから始める異世界生活""","""TV""","""25""","""Apr 4, 2016 to Sep 19, 2016""","""TV Tokyo, Media Factory, Kadokawa Shoten, AT-X, Magic Capsule, Memory-Tech, Hakuhodo DY Music & Pict…","""Funimation""","""White Fox""","""Light novel""","""25 min. per ep.""","""R - 17+ (violence & profanity)""",1478842,106570,1130438,30627,34941,176266
